In [1]:
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np

import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
import jax
import jax.numpy as jnp
import flax.linen as nn
import networkx as nx
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np
from physnetjax.models.model import EF
from physnetjax.training.loss import dipole_calc
from  physnetjax.training.training import train_model # from model import dipole_calc
from physnetjax.data.data import prepare_batches, prepare_datasets
import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
from pathlib import Path
# from physnetjax.analysis import *
import polars as pl
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorflow.python.summary.summary_iterator import summary_iterator

In [2]:

data_key, train_key = jax.random.split(jax.random.PRNGKey(42), 2)


NATOMS = 30
files = [Path('/pchem-data/meuwly/boittier/home/pycharmm_test/data/basepairs/at_prod.npz'),
 Path('/pchem-data/meuwly/boittier/home/pycharmm_test/data/basepairs/at_reag.npz'),
 Path('/pchem-data/meuwly/boittier/home/pycharmm_test/data/basepairs/at_retune.npz'),
 Path('/pchem-data/meuwly/boittier/home/pycharmm_test/data/basepairs/rattle_neb_at.npz'),
 Path('/pchem-data/meuwly/boittier/home/pycharmm_test/data/basepairs/rattle_neb_gc.npz')]


train_data, valid_data = prepare_datasets(
    data_key,
    1220,
    20,
    files,
    clip_esp=False,
    natoms=NATOMS,
    clean=False,
    subtract_atom_energies=True,
    verbose=True,
    #    data_key, 27, 20, files, clip_esp=False, natoms=NATOMS, clean=False
)

ntest = len(valid_data["E"]) // 2
test_data = {k: v[ntest:] for k, v in valid_data.items()}
valid_data = {k: v[:ntest] for k, v in valid_data.items()}


In [3]:
data_keys = ("R", "Z", "F", "E", "D", "N", "dst_idx", "src_idx", "batch_segments")

batch_size = 10
test_batches = prepare_batches(data_key, test_data, batch_size,
                              num_atoms=NATOMS, 
                              data_keys=data_keys)

train_batches = prepare_batches(data_key, train_data, batch_size,
                              num_atoms=NATOMS, 
                              data_keys=data_keys)

valid_batches = prepare_batches(data_key, valid_data, batch_size,
                              num_atoms=NATOMS, 
                              data_keys=data_keys)

# params = pd.read_pickle("/pchem-data/meuwly/boittier/home/pycharmm_test/checkpoints/diox-q3.2.pkl")
# for _ in test_batches[0].keys():
#     print(_, test_batches[0][_].shape)
# valid_batches

In [4]:
# valid_batches[0]

In [5]:
combined = test_batches+ valid_batches #+ train_batches
len(combined)

In [6]:

from physnetjax.restart.restart import get_last, get_files, get_params_model
from physnetjax.analysis.analysis import plot_stats
restart = "/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/efa0basepairs-1619ffed-2265-42ff-991a-37adcb30ff3c/"
restart = get_last(restart)

params,model = get_params_model(restart)
model.natoms = 30
print(model)

output = plot_stats(combined[:], model, params, _set="Test", 
               do_kde=False, batch_size=batch_size)

In [141]:
from ebc.Clustering import EBC
energies = np.array([_["E"] for _ in combined])
atomic_numbers = np.array([_["Z"] for _ in combined])

In [142]:
plt.plot(np.arange(energies.shape[0]), energies.flatten(), alpha=0.1)
plt.scatter(np.arange(energies.shape[0]), energies.flatten(), alpha=0.5, s=0.51)

In [143]:
plt.hist(energies.flatten())

In [144]:
from physnetjax.analysis.analysis  import remove_mean_from_multimodal_distribution

energies_mean_removed = remove_mean_from_multimodal_distribution(energies.flatten())

plt.hist(energies_mean_removed)

In [145]:
import seaborn as sns
sns.stripplot(energies_mean_removed, c=ebc.state_labels, cmap="Set2", jitter=0.4, dodge=True, orient="h")


In [146]:
dir(ebc)
# ebc.proto_labels
ebc.state_labels.shape

In [178]:
from physnetjax.analysis.analysis import *
from physnetjax.analysis.povray_tool import *

In [218]:
species = np.unique(np.concatenate(atomic_numbers))
# Apply PCA
labels = None
soap_descr, ase_atoms = compute_soap_descriptors(
    coordinates, atomic_numbers, species, 
    r_cut=10.0, n_max=16, l_max=4, sigma=1.0)

In [219]:
reduced_data_pca, model_pca = process_data(soap_descr, apply_pca, n_components=2)

In [220]:
visualize_projection(reduced_data_pca, labels,
                     title="PCA Projection", c=energies_mean_removed)

In [223]:
n = 4
ebc = EBC(n_clusters=n) 
labels = ebc.fit_transform(soap_descr[:,:], energies)
_ = ebc.show()
# plt.show()
cluster_members_energy = {_:float(energies_mean_removed[_]) for _ in range(int(ebc._proto_2D.shape[0]))}
_cluster_members_energy = np.array([v for v in 
                                    cluster_members_energy.values()])
# plt.scatter(ebc._proto_2D[:, 0], ebc._proto_2D[:, 1], alpha=0.1, c=errors, zorder=-1)
s = plt.scatter(ebc._proto_2D[:, 0][::], ebc._proto_2D[:, 1][::], alpha=1, c=[cluster_members_energy[_]
                for _ in range(int(ebc._proto_2D.shape[0]))], 
                vmin=_cluster_members_energy.min(), vmax=_cluster_members_energy.max(), 
                zorder=-1, cmap="jet",s=10)
plt.colorbar(s)
plt.show()

In [222]:
annotate_ebc(ebc, energies, ase_atoms)